## ETL -1

In [1]:
# librerias:
import requests
import pandas as pd
import numpy as np

# para ver más de una salida por celda
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

En el marco de un proyecto de análisis de datos, se requiere realizar una ETL para integrar información de distintas fuentes de datos. La empresa para la que trabajamos cuenta con una base de datos de la informacion sobre el número de personas censadas en las distintas Comunidades Autónomas españolas desde el año 1998 hasta el 2022. Quieren enriquecer los datos con información de la generación de energías renovables y no renovables y crear una base de datos más completa. Para esto nos piden que usemos la API de REData la cual nos da mucha información sobre el mercado energético en España. La documentación de la API la encontraréis aquí:

https://www.ree.es/es/apidatos

In [3]:
# definimos la url de la API
# url = 'https://www.ree.es/es/apidatos'
url = 'https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date=2011-01-01T00:00&end_date=2011-12-31T23:59&time_trunc=year'

In [43]:
# hacemos la petición de datos a la API
response = requests.get(url = url)

In [44]:
response

<Response [200]>

In [5]:
# comprobamos si la petición se está realizando correctamente
r_stat = response.status_code
r_reason = response.reason

print(f'El código devuelto por la API es {r_stat}, con razón: {r_reason}')

El código devuelto por la API es 200, con razón: OK


Requisitos de este ejercicio de pair programming:

1- Nuestros jefes están especialmente interesados en los datos de generación de las energías renovables y no renovables a nivel nacional. En concreto, nos piden sacar toda la información para las fechas de 1 de Enero del 2011 hasta el 31 de Diciembre del 2022. De toda la información que tenemos de generación queremos la evolucion de renovable y no renovable. Además deberemos incluir una columna nueva identificando si el dato corresponde a energía renovable o no renovable.

In [22]:
df_final = pd.DataFrame(response.json()['included'])
df_final

,type,id,groupId,attributes
0,Renovable,10299,None,"{'title': 'Renovable', 'description': 'Energía..."
1,No renovable,10298,None,"{'title': 'No renovable', 'description': None,..."


In [47]:
renovable = response.json()['included'][0]['attributes']['values']
renovable

[{'value': 86667052.8500005,
  'percentage': 0.31024115642116423,
  'datetime': '2011-01-01T00:00:00.000+01:00'}]

In [24]:
renovable = response.json()['included'][0]
no_renovable = response.json()['included'][1]

In [34]:
pd.DataFrame(renovable['attributes']['values'])

,value,percentage,datetime
0,8.666705e+07,0.310241,2011-01-01T00:00:00.000+01:00


In [49]:
df_final = pd.DataFrame()
df_renov = pd.DataFrame()
df_no_renov = pd.DataFrame()

In [50]:
for año in range(2011,2023):
    url = f'https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=year'
    response = requests.get(url = url)

    renovables = pd.DataFrame(response.json()['included'][0]['attributes']['values'])
    df_renov = pd.concat([df_renov, renovables],  axis=0, ignore_index = True)
    df_renov['type'] = 'renovable'
    
    
    no_renovables = pd.DataFrame(response.json()['included'][1]['attributes']['values'])
    df_no_renov = pd.concat([df_no_renov, no_renovables],  axis=0, ignore_index = True)
    df_no_renov['type'] = 'no renovable'

df_final = pd.concat([df_renov, df_no_renov],  axis=0, ignore_index = True)

In [51]:
df_final

,value,percentage,datetime,type
0,8.666705e+07,0.310241,2011-01-01T00:00:00.000+01:00,renovable
1,8.533872e+07,0.301424,2012-01-01T00:00:00.000+01:00,renovable
2,1.097577e+08,0.401622,2013-01-01T00:00:00.000+01:00,renovable
3,1.078761e+08,0.404771,2014-01-01T00:00:00.000+01:00,renovable
4,9.408811e+07,0.351792,2015-01-01T00:00:00.000+01:00,renovable
5,1.010892e+08,0.386079,2016-01-01T00:00:00.000+01:00,renovable
6,8.461087e+07,0.322566,2017-01-01T00:00:00.000+01:00,renovable
7,1.003444e+08,0.384488,2018-01-01T00:00:00.000+01:00,renovable
8,9.791330e+07,0.375393,2019-01-01T00:00:00.000+01:00,renovable
9,1.106052e+08,0.439959,2020-01-01T00:00:00.000+01:00,renovable


2- Sacaremos la misma información que en el ejericio anterior, pero en este caso por CCAA. Para esto necesitaremos los códigos de cada una de las comunidades autónomas que encontraréis en el siguiente diccionario:

In [53]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [78]:
for k,v in cod_comunidades.items():
    print(k)

Ceuta
Melilla
Andalucía
Aragón
Cantabria
Castilla - La Mancha
Castilla y León
Cataluña
País Vasco
Principado de Asturias
Comunidad de Madrid
Comunidad Foral de Navarra
Comunitat Valenciana
Extremadura
Galicia
Illes Balears
Canarias
Región de Murcia
La Rioja


In [79]:
df_final_ccaa = pd.DataFrame()
df_renov2 = pd.DataFrame()
df_no_renov2 = pd.DataFrame()

In [98]:
for k,v in cod_comunidades.items():
    url_ccaa = f'https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date=2011-01-01T00:00&end_date=2011-12-31T23:59&time_trunc=year&geo_limit=ccaa&geo_ids={v}'
    response2 = requests.get(url = url_ccaa)

    renovables2 = pd.DataFrame(response2.json()['included'][0]['attributes']['values'])
    df_renov2 = pd.concat([df_renov2, renovables2],  axis=0, ignore_index = True)
    df_renov2[['type', 'CCAA']] = ['renovable', k]

    
    no_renovables2 = pd.DataFrame(response2.json()['included'][1]['attributes']['values'])
    df_no_renov2 = pd.concat([df_no_renov2, no_renovables2],  axis=0, ignore_index = True)
    df_no_renov2[['type', 'CCAA']] = ['no renovable', k]

df_final_ccaa = pd.concat([df_renov2, df_no_renov2],  axis=0, ignore_index = True)

IndexError: list index out of range

In [99]:
url_ccaa = f'https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date=2011-01-01T00:00&end_date=2011-12-31T23:59&time_trunc=year&geo_limit=ccaa&geo_ids=7'
response2 = requests.get(url = url_ccaa)

In [100]:
response2.json()['included']

[{'type': 'Renovable',
  'id': '10343',
  'groupId': None,
  'attributes': {'title': 'Renovable',
   'description': 'Energía renovable: incluye hidráulica, hidroeólica, eólica, solar fotovoltaica, solar térmica, otras renovables y residuos renovables.',
   'color': '#92d050',
   'type': 'Renovable',
   'magnitude': None,
   'composite': False,
   'last-update': '2019-06-20T14:30:27.000+02:00',
   'values': [{'value': 9874557.135,
     'percentage': 0.410912377544615,
     'datetime': '2011-01-01T00:00:00.000+01:00'}]}},
 {'type': 'No renovable',
  'id': '10342',
  'groupId': None,
  'attributes': {'title': 'No renovable',
   'description': None,
   'color': '#666666',
   'type': 'No renovable',
   'magnitude': None,
   'composite': False,
   'last-update': '2019-06-20T14:30:27.000+02:00',
   'values': [{'value': 14156252.533,
     'percentage': 0.589087622455385,
     'datetime': '2011-01-01T00:00:00.000+01:00'}]}}]

In [91]:
response2.json()['included'][0]['attributes']['values']

[{'value': 202972.893,
  'percentage': 1,
  'datetime': '2011-01-01T00:00:00.000+01:00'}]

In [ ]:
df_final_ccaa

In [77]:
k

,value,percentage,datetime
0,1.165831e+08,0.421921,2022-01-01T00:00:00.000+01:00


In [74]:
response2.json()['included'][0]['attributes']['values']

[{'value': 116583131.8090003,
  'percentage': 0.42192065521696676,
  'datetime': '2022-01-01T00:00:00.000+01:00'}]

In [ ]:
'https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date=2014-01-01T00:00&end_date=2018-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids=7'

In [58]:
response2.json()['included']

[{'type': 'Renovable',
  'id': '10299',
  'groupId': None,
  'attributes': {'title': 'Renovable',
   'description': 'Energía renovable: incluye hidráulica, hidroeólica, eólica, solar fotovoltaica, solar térmica, otras renovables y residuos renovables.',
   'color': '#92d050',
   'type': 'Renovable',
   'magnitude': None,
   'composite': False,
   'last-update': '2022-12-12T12:48:47.000+01:00',
   'values': [{'value': 116583131.8090003,
     'percentage': 0.42192065521696676,
     'datetime': '2022-01-01T00:00:00.000+01:00'}]}},
 {'type': 'No renovable',
  'id': '10298',
  'groupId': None,
  'attributes': {'title': 'No renovable',
   'description': None,
   'color': '#666666',
   'type': 'No renovable',
   'magnitude': None,
   'composite': False,
   'last-update': '2022-12-12T12:48:43.000+01:00',
   'values': [{'value': 159732166.7370002,
     'percentage': 0.5780793447830334,
     'datetime': '2022-01-01T00:00:00.000+01:00'}]}}]

In [54]:
for k,v in cod_comunidades.items():
    
    for año in range(2011,2023):
    url_ccaa = f'https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={v}'
    response2 = requests.get(url = url)

    renovables = pd.DataFrame(response.json()['included'][0]['attributes']['values'])
    df_renov = pd.concat([df_renov, renovables],  axis=0, ignore_index = True)
    df_renov['type'] = 'renovable'
    
    
    no_renovables = pd.DataFrame(response.json()['included'][1]['attributes']['values'])
    df_no_renov = pd.concat([df_no_renov, no_renovables],  axis=0, ignore_index = True)
    df_no_renov['type'] = 'no renovable'


Ceuta 8744
Melilla 8745
Andalucía 4
Aragón 5
Cantabria 6
Castilla - La Mancha 7
Castilla y León 8
Cataluña 9
País Vasco 10
Principado de Asturias 11
Comunidad de Madrid 13
Comunidad Foral de Navarra 14
Comunitat Valenciana 15
Extremadura 16
Galicia 17
Illes Balears 8743
Canarias 8742
Región de Murcia 21
La Rioja 20


In [ ]:
url_ccaa= 'https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={ccaa_id}'

In [ ]:
for año in range(2011,2023):
    url = f'https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=year'
    response = requests.get(url = url)

    renovables = pd.DataFrame(response.json()['included'][0]['attributes']['values'])
    df_renov = pd.concat([df_renov, renovables],  axis=0, ignore_index = True)
    df_renov['type'] = 'renovable'
    
    
    no_renovables = pd.DataFrame(response.json()['included'][1]['attributes']['values'])
    df_no_renov = pd.concat([df_no_renov, no_renovables],  axis=0, ignore_index = True)
    df_no_renov['type'] = 'no renovable'

df_final = pd.concat([df_renov, df_no_renov],  axis=0, ignore_index = True)

3- Guardad los csv ya que mañana los seguiremos usando en los siguientes pairs